In [1]:
cd("/home/jzh/Dropbox/Research/Data-driven_estimation_inverse_optimization/Experiments/InverseVIsTraffic");

In [2]:
type Arc
    initNode::Int 
    termNode::Int 
    capacity::Float64
    freeflowtime::Float64
            
            
    trueflow::Float64
    obsflow::Float64
end

Arc(initNode::Int, termNode::Int, capacity::Float64,freeflowtime::Float64) = 
    Arc(initNode, termNode, capacity, freeflowtime, 0., 0.)

Arc (constructor with 2 methods)

In [3]:
using JuMP
using Gurobi
using Graphs
using Roots

polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)])  #VG think about faster way to do this
polyEval(coeffs::Array{Float64, 1}, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) #separate for consts

bpacost(flow::Float64, capacity::Float64, freeflowtime::Float64) = freeflowtime*(1 + .15 * (flow/capacity)^4)
bpacost(flow::Float64, arc) = bpacost(flow, arc.capacity, arc.freeflowtime)
bpacost(arc::Arc) = bpacost(arc.obsflow, arc)

LoadError: JuMP not found
at In[3]:1

In [4]:
#Uses a Frank-Wolfe algorithm to solve bpa cost for the given network.
#construct the underlying graph
#Fix an ordering of the arcs... should just be pointers
function frank_wolfe(g, vArcs, demand_data, idx; TOL=1e-4, MAX_ITERS=100)
    #use the observed flows as the starting point
    flows =[a.obsflow::Float64 for a in vArcs]
    print("Initially, we have: \n\n")
    print("flows = ", "\n", flows, "\n\n")
    costs = [bpacost(a) for a in vArcs]
    print("costs = ", "\n", costs, "\n\n")
    trace = Float64[]
    print("trace = ", "\n", trace, "\n\n")
    print("==============================================================", "\n")
    for iter = 1:MAX_ITERS
        print("Iteration times: $iter \n\n")
        flow_dict = Dict{(Int, Int), Float64}()
        for odpair = keys(demand_data)
            #solve the shortest path problems, and update the total flow
            r = dijkstra_shortest_paths(g, costs, odpair[1] )
            currNode = odpair[2];
            while currNode != odpair[1]
                parent = r.parents[currNode]
                if ! haskey(flow_dict, (parent, currNode) )
                    flow_dict[(parent, currNode)] = demand_data[odpair][idx]  
                else
                    flow_dict[(parent, currNode)] += demand_data[odpair][idx]  
                end
                currNode = parent
            end
        end

        d = [get(flow_dict, (a.initNode, a.termNode), 0.)::Float64 for a in vArcs]
        print("d: $d \n\n")
        
        #In the first iteration, just pull out the flows
        if iter == 1
        	flows = d
        	costs = [bpacost(flows[ix], a) for (ix, a) in enumerate(vArcs)]
        	continue
        end
        assert( dot(costs, d) <= dot(flows, costs) )
        d -= flows 
        derivFun(alpha) = sum([bpacost(flows[ix] + alpha*d[ix], a)*d[ix] for (ix, a) in enumerate(vArcs)])
        if derivFun(0) >=0 
            alpha = 0
        elseif derivFun(1) <= 0
            alpha = 1
        else
            alpha = fzero(derivFun, 0, 1)
        end
        print("alpha: $alpha \n\n")
        converge_dist = alpha * norm(d) / norm(flows)
        flows += alpha * d
        push!(trace, converge_dist)
        print("trace: $trace \n\n")
        print("--------------------------------------------------------------", "\n")
        if (iter > 1) & (converge_dist <= TOL)
            break
        else
            costs = [bpacost(flows[ix], a) for (ix, a) in enumerate(vArcs)]
        end
        
    end
    # VG Consider doing a polishing step to refine solution at the end...
    # Maybe solve a simplicial assignment on last several extreme pt solutions via 1st order method
    # Maybe switch over to refined step determination from fukushima paper
    return trace[length(trace)], flows
end

frank_wolfe (generic function with 1 method)

In [57]:
############
#Read in the demand file
file = open("./data_small_size/SiouxFalls_trips.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand
            end
        end
    end
end                
close(file)

In [58]:
############
#read in the arc files
arcs = Dict{(Int, Int), Arc}()
file = open("./data_small_size/SiouxFalls_net.txt")
inHeader=true
for line in eachline(file)
    if inHeader
        inHeader = !contains(line, "Init node")
        continue
    end
    vals = split(line, )
    arcs[(int(vals[1]), int(vals[2]))] = Arc(int(vals[1]), int(vals[2]), 
                                              float(vals[3]), float(vals[5]))
end
close(file)

In [59]:
###########
#read in the initial flows
file = open("./data_small_size/SiouxFallsFlow.txt")
ix = 0; 
for line in eachline(file)
    ix +=1
    if ix ==1
        continue
    end
    vals = split(line)
    arcs[(int(vals[1]), int(vals[2]))].trueflow = float(vals[3])
end
close(file)

In [60]:
##########
#Generate the simulated data
##########
numData = 1; sigma = .0
flow_data = Array(Float64, length(arcs), numData)
demand_data = Dict{(Int, Int), Array{Float64, 1}}()

numNodes = maximum(map(pair->pair[1], keys(demands)))
g = simple_inclist(numNodes, is_directed=true)
vArcs = Arc[]
for arc in values(arcs)
    arc.obsflow = arc.trueflow
    add_edge!(g, arc.initNode, arc.termNode) 
    push!(vArcs, arc)
end

#srand(8675309)
for iRun = 1:numData
    #perturb the demand_data
    for odpair in keys(demands)
        if ! haskey(demand_data, odpair)
            demand_data[odpair] = [demands[odpair] * (1 + sigma * rand()), ]
        else
            push!(demand_data[odpair], demands[odpair] * (1  + sigma * rand()))
        end
    end
    #solve using FW and record
    conv_tol, flow_data[:, iRun] = frank_wolfe(g, vArcs, demand_data, iRun)
    print("conv_tol = ", "\n", conv_tol, "\n\n", "flow_data = ", "\n", flow_data, "\n\n")
end

#Randomzie the flow data a little bit too
for i = 1:size(flow_data, 1)
    for j = 1:size(flow_data, 2)
        flow_data[i, j] *= (1 + sigma * rand() )
    end
end

Initially, we have: 

flows = 
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

costs = 
[4.0,5.0,4.0,2.0,2.0,6.0,10.0,4.0,5.0,5.0,2.0,2.0,4.0,3.0,3.0,3.0,4.0,6.0,10.0,3.0,4.0,5.0]

trace = 
[]

Iteration times: 1 

d: [3800.0,3700.0,5300.0,5200.0,7200.0,1800.0,4900.0,2700.0,7900.0,3700.0,5200.0,7200.0,2700.0,5300.0,5300.0,11500.0,5300.0,1800.0,4900.0,11500.0,3800.0,7900.0] 

Iteration times: 2 

d: [3800.0,3700.0,5300.0,5200.0,7200.0,1800.0,4900.0,2700.0,7900.0,3700.0,5200.0,7200.0,2700.0,5300.0,5300.0,11500.0,5300.0,1800.0,4900.0,11500.0,3800.0,7900.0] 

alpha: 0 

trace: [0.0] 

--------------------------------------------------------------
conv_tol = 
0.0

flow_data = 
[3800.0
 3700.0
 5300.0
 5200.0
 7200.0
 1800.0
 4900.0
 2700.0
 7900.0
 3700.0
 5200.0
 7200.0
 2700.0
 5300.0
 5300.0
 11500.0
 5300.0
 1800.0
 4900.0
 11500.0
 3800.0
 7900.0]



In [51]:
arcs

Dict{(Int64,Int64),Arc} with 6 entries:
  (1,2) => Arc(1,2,10000.0,6.0,0.0,0.0)
  (3,1) => Arc(3,1,30000.0,4.0,0.0,0.0)
  (2,3) => Arc(2,3,500.0,100.0,0.0,0.0)
  (3,2) => Arc(3,2,500.0,100.0,0.0,0.0)
  (1,3) => Arc(1,3,30000.0,4.0,0.0,0.0)
  (2,1) => Arc(2,1,10000.0,6.0,0.0,0.0)

In [56]:
flow_data

22x1 Array{Float64,2}:
  3800.0
  3700.0
  5300.0
  5200.0
  7200.0
  1800.0
  4900.0
  2700.0
  7900.0
  3700.0
  5200.0
  7200.0
  2700.0
  5300.0
  5300.0
 11500.0
  5300.0
  1800.0
  4900.0
 11500.0
  3800.0
  7900.0

In [61]:
flow_data

22x1 Array{Float64,2}:
  3800.0
  3700.0
  5300.0
  5200.0
  7200.0
  1800.0
  4900.0
  2700.0
  7900.0
  3700.0
  5200.0
  7200.0
  2700.0
  5300.0
  5300.0
 11500.0
  5300.0
  1800.0
  4900.0
 11500.0
  3800.0
  7900.0

In [1]:
M = [[2,0,0,0,3,0] [0,2,0,0,0,3] [0,0,5,0,0,0] [0,0,0,5,0,0] [0.8,0,0,0,2,0] [0,0.8,0,0,0,2]]

6x6 Array{Float64,2}:
 2.0  0.0  0.0  0.0  0.8  0.0
 0.0  2.0  0.0  0.0  0.0  0.8
 0.0  0.0  5.0  0.0  0.0  0.0
 0.0  0.0  0.0  5.0  0.0  0.0
 3.0  0.0  0.0  0.0  2.0  0.0
 0.0  3.0  0.0  0.0  0.0  2.0

In [2]:
J=(M+M')/2

6x6 Array{Float64,2}:
 2.0  0.0  0.0  0.0  1.9  0.0
 0.0  2.0  0.0  0.0  0.0  1.9
 0.0  0.0  5.0  0.0  0.0  0.0
 0.0  0.0  0.0  5.0  0.0  0.0
 1.9  0.0  0.0  0.0  2.0  0.0
 0.0  1.9  0.0  0.0  0.0  2.0

In [3]:
eig(J)

([0.1,0.1,3.9,3.9,5.0,5.0],
6x6 Array{Float64,2}:
  0.0        0.707107      0.707107      0.0        0.0          -2.33731e-16
 -0.707107   2.22045e-16  -2.22045e-16  -0.707107   0.0           0.0        
  0.0        0.0           0.0           0.0        1.0           2.33731e-16
  0.0        1.11022e-16   1.11022e-16   0.0       -2.33731e-16   1.0        
  0.0       -0.707107      0.707107      0.0        0.0           0.0        
  0.707107   0.0           0.0          -0.707107   0.0           0.0        )

In [11]:
A = [[1,0,0,0,1.5,0] [0,1,0,0,0,1.5] [0,0,1,0,0,0] [0,0,0,1,0,0] [1.5,0,0,0,2.25,0] [0,1.5,0,0,0,2.25]]

6x6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  1.5   0.0 
 0.0  1.0  0.0  0.0  0.0   1.5 
 0.0  0.0  1.0  0.0  0.0   0.0 
 0.0  0.0  0.0  1.0  0.0   0.0 
 1.5  0.0  0.0  0.0  2.25  0.0 
 0.0  1.5  0.0  0.0  0.0   2.25

In [12]:
eig(A)

([1.33227e-15,1.33227e-15,1.0,1.0,3.25,3.25],
6x6 Array{Float64,2}:
  0.83205   0.0      0.0  0.0  -0.5547   0.0    
  0.0      -0.83205  0.0  0.0   0.0      0.5547 
  0.0       0.0      1.0  0.0   0.0      0.0    
  0.0       0.0      0.0  1.0   0.0      0.0    
 -0.5547    0.0      0.0  0.0  -0.83205  0.0    
  0.0       0.5547   0.0  0.0   0.0      0.83205)

In [1]:
deg = 3
c = 1.5
samples = linspace(0, 1, deg + 1)
k(x,y) = (c + x*y)^deg
K = [ k(x,y) for x = samples, y=samples]
K = convert(Array{Float64, 2}, K)
assert(rank(K) == deg+1)

In [2]:
K

4x4 Array{Float64,2}:
 3.375  3.375     3.375     3.375  
 3.375  4.18193   5.1082    6.16204
 3.375  5.1082    7.35168  10.1713 
 3.375  6.16204  10.1713   15.625  

In [3]:
rank(K)

4

In [4]:
C = chol(K + 1e-6* eye(deg+1))

4x4 Array{Float64,2}:
 1.83712  1.83712   1.83712   1.83712 
 0.0      0.898292  1.92944   3.1026  
 0.0      0.0       0.503946  1.60739 
 0.0      0.0       0.0       0.200469

In [1]:
using JuMP
using Gurobi
using Graphs
using Roots

deg = 3
m = Model(solver=GurobiSolver(OutputFlag=false))
@defVar(m, Calphas[1:deg+1])
@defVar(m, reg_term >= 0)
reg_term_ = QuadExpr(Calphas[:], Calphas[:], ones(deg+1), AffExpr() )
@addConstraint(m, reg_term >= reg_term_)


-Calphas[1]² - Calphas[2]² - Calphas[3]² - Calphas[4]² + reg_term ≥ 0

In [2]:
AffExpr()

0

In [30]:
using JuMP
using Gurobi
using Graphs
using Roots


polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)])  #VG think about faster way to do this
polyEval(coeffs::Array{Float64, 1}, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) #separate for consts

bpacost(flow::Float64, capacity::Float64, freeflowtime::Float64) = freeflowtime*(1 + .15 * (flow/capacity)^4)


function setUpFitting(deg::Int, c)
	m = Model(solver=GurobiSolver(OutputFlag=false))
	@defVar(m, coeffs[1:deg+1])
	@defVar(m, Calphas[1:deg+1])

	##VG Probably want to go back and redo this with an intercept term
	#build the graham matrix
	samples = linspace(0, 1, deg + 1)
	k(x,y) = (c + x*y)^deg
	K = [ k(x,y) for x = samples, y=samples]
	K = convert(Array{Float64, 2}, K)
	assert(rank(K) == deg+1)
    
    # C是一个上三角阵，满足: C' * C = K
	C = chol(K + 1e-6* eye(deg+1))
    
	for i=1:deg + 1
		@addConstraint(m, polyEval(coeffs, samples[i]) == 
						sum{C[j, i] * Calphas[j], j=1:deg+1})
	end
	@defVar(m, reg_term >= 0)
	reg_term_ = QuadExpr(Calphas[:], Calphas[:], ones(deg+1), AffExpr() )
	@addConstraint(m, reg_term >= reg_term_)

	return m, coeffs, reg_term, C, K
end

setUpFitting (generic function with 2 methods)

In [31]:
deg = 2
c = 1
m, coeffs, reg_term, C, K = setUpFitting(deg, c)

(Feasibility problem with:
 * 3 linear constraints
 * 1 quadratic constraint
 * 7 variables
Solver set to Gurobi,coeffs[i] free for all i in {1,2,3},reg_term,
3x3 Array{Float64,2}:
 1.0  1.0       1.0     
 0.0  0.750001  1.66667 
 0.0  0.0       0.471412,

3x3 Array{Float64,2}:
 1.0  1.0     1.0 
 1.0  1.5625  2.25
 1.0  2.25    4.0 )

In [32]:
m

Feasibility problem with:
 * 3 linear constraints
 * 1 quadratic constraint
 * 7 variables
Solver set to Gurobi

In [33]:
coeffs

coeffs[i] free for all i in {1,2,3}

In [34]:
reg_term

reg_term

In [35]:
C

3x3 Array{Float64,2}:
 1.0  1.0       1.0     
 0.0  0.750001  1.66667 
 0.0  0.0       0.471412

In [36]:
K

3x3 Array{Float64,2}:
 1.0  1.0     1.0 
 1.0  1.5625  2.25
 1.0  2.25    4.0 

In [37]:
C*C'

3x3 Array{Float64,2}:
 3.0       2.41667   0.471412
 2.41667   3.34027   0.785687
 0.471412  0.785687  0.22223 

In [38]:
C'

3x3 Array{Float64,2}:
 1.0  0.0       0.0     
 1.0  0.750001  0.0     
 1.0  1.66667   0.471412

In [40]:
C' * C

3x3 Array{Float64,2}:
 1.0  1.0     1.0 
 1.0  1.5625  2.25
 1.0  2.25    4.0 

In [5]:
x = 3

3

In [6]:
x = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [7]:
y = [1 2 3]

1x3 Array{Int64,2}:
 1  2  3

In [14]:
-p 2

LoadError: syntax: extra token "2" after end of expression
while loading In[14], in expression starting on line 1

In [3]:
using TrafficAssignment
using Base.Test

In [4]:
# write your own tests here
data_time = time()
ta_data = load_ta_network("Sioux Falls")

TA_Data("Sioux Falls",24,24,1,76,[1,1,2,2,3,3,3,4,4,4  …  22,22,22,22,23,23,23,24,24,24],[2,3,1,6,1,4,12,3,5,11  …  15,20,21,23,14,22,24,13,21,23],[25900.2,23403.5,25900.2,4958.18,23403.5,17110.5,23403.5,17110.5,17782.8,4908.83  …  9599.18,5075.7,5229.91,5000.0,4924.79,5000.0,5078.51,5091.26,4885.36,5078.51],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15  …  0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15],[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0  …  4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,1,1,1,1,1,1],360600.0,24x24 Array{Float64,2}:
    0.0  100.0  100.0   500.0   200.0  …   100.0   400.0   300.0   100.0
  

In [5]:
println("Data Loading Completed, time:", time() - data_time, " seconds")

Data Loading Completed, time:11.369055032730103 seconds


In [8]:
link_flow, link_travel_time, objective = ta_frank_wolfe(ta_data, method=:bfw, step=:exact, log=:on, tol=1e-5, max_iter_no=2000)

-------------------------------------
Network Name: Sioux Falls
Method: bfw
Line Search Step: exact
Maximum Interation Number: 2000
Tolerance for AEC: 1.0e-5
Number of processors: 1
Setup time = 0.013502120971679688 seconds
k=1,	tauk=0.3199426617213497,	objective=1.6368800421965161e7,	aec=172.76814835715197
k=2,	tauk=0.2445967720763769,	objective=7.121451141877039e6,	aec=31.05136201349762
k=3,	tauk=0.19907062649129206,	objective=5.816530837659503e6,	aec=9.609754845979461
k=4,	tauk=0.8655087931429251,	objective=5.468680976781959e6,	aec=6.037390199544071
k=5,	tauk=0.9999999814878309,	objective=4.90637856331669e6,	aec=6.380174991147677
k=6,	tauk=0.20521830598638904,	objective=4.789330576124015e6,	aec=5.10642826044441
k=7,	tauk=0.2454941147262086,	objective=4.603650060534018e6,	aec=2.0136478267985933
k=8,	tauk=0.5123132848338909,	objective=4.518965480833182e6,	aec=1.439909496842868
k=9,	tauk=0.9999999814878309,	objective=4.441919867581811e6,	aec=1.4335873081349715
k=10,	tauk=0.111998383095

([4494.65,8119.04,4519.04,5967.35,8094.65,14006.6,10022.5,14030.8,18006.6,5200.0  …  18386.1,7000.0,8607.17,9661.84,8394.72,9626.21,7902.66,11112.2,10259.5,7861.48],[6.00082,4.00869,6.00083,6.57361,4.00859,4.26942,4.02018,4.27129,2.31539,7.1333  …  9.05662,7.71313,4.20083,12.3658,9.06552,12.2431,3.75902,17.6163,11.7525,3.72264],4.231335593540048e6)

In [7]:
link_flow

76-element Array{Float64,1}:
  4494.98
  8118.91
  4519.21
  5967.77
  8094.68
 14012.9 
 10029.8 
 14037.3 
 18012.0 
  5204.89
 18036.6 
  8796.62
 15784.7 
     ⋮   
  8619.4 
 10307.8 
 18384.7 
  7000.9 
  8606.8 
  9660.47
  8393.37
  9625.48
  7902.31
 11111.2 
 10258.0 
  7861.46

In [16]:
ta_data.network_name

"Sioux Falls"

In [18]:
ta_data.capacity

76-element Array{Float64,1}:
 25900.2 
 23403.5 
 25900.2 
  4958.18
 23403.5 
 17110.5 
 23403.5 
 17110.5 
 17782.8 
  4908.83
 17782.8 
  4948.0 
 10000.0 
     ⋮   
  5229.91
  4885.36
  9599.18
  5075.7 
  5229.91
  5000.0 
  4924.79
  5000.0 
  5078.51
  5091.26
  4885.36
  5078.51